In [1]:
import seaborn as sns
import string
import re
from collections import Counter
import matplotlib.pyplot as plt
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load(disable=['parser','tagger','ner'])
import pandas as pd

In [2]:
import string
punctuations = string.punctuation
punctuations=punctuations+'—'+'–'+'--'

In [3]:
# Creating our tokenizer function
def spacy_tokenizer(text):
    # Creating our token object, which is used to create documents with linguistic annotations.
    mytokens = parser(text)

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.lower().strip() if word.lemma_ !=
                "-PRON-" else word.lower_ for word in mytokens]

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words]
    
    #Removing punctuations
    mytokens = [word for word in mytokens if word not in punctuations]

    mytokens = ' '.join(mytokens)
    # return preprocessed list of tokens
    return mytokens

In [4]:
campaign_speeches=pd.read_csv('Campaign Speeches')

In [5]:
campaign_speeches.head()

,candidate,date,speech title,text,party,won_nomination,sentiment_score,positive,negative
0,Robert Dole,1995-04-10,Remarks Announcing Candidacy for the Republica...,"Thank you, Governor Graves, Senator Kassebaum....",Republican,1,"{'neg': 0.088, 'neu': 0.736, 'pos': 0.175, 'co...",0.175,0.088
1,Robert Dole,1995-05-31,"Remarks in Los Angeles: ""Hollywood Speech""",I want to talk about a specific matter tonight...,Republican,1,"{'neg': 0.143, 'neu': 0.657, 'pos': 0.2, 'comp...",0.200,0.143
2,Robert Dole,1995-06-17,Remarks to the United States Conference of May...,"Thank you, Senator Kempthorne, and thank you a...",Republican,1,"{'neg': 0.12, 'neu': 0.711, 'pos': 0.17, 'comp...",0.170,0.120
3,Robert Dole,1995-08-19,Remarks at the Iowa Straw Poll in Ames,"Thank you, Dan Quayle , for that great introdu...",Republican,1,"{'neg': 0.103, 'neu': 0.717, 'pos': 0.181, 'co...",0.181,0.103
4,Robert Dole,1995-09-04,Remarks at the American Legion Convention in I...,"Thank you, and thank you very much, Commander ...",Republican,1,"{'neg': 0.103, 'neu': 0.725, 'pos': 0.172, 'co...",0.172,0.103


In [6]:
campaign_speeches=campaign_speeches.dropna(subset=['text'])

In [7]:
campaign_speeches.shape

(1691, 9)

In [9]:
party_speeches = campaign_speeches[(campaign_speeches.party == 'Democrat') & (campaign_speeches.won_nomination == 1)].reset_index(drop=True)

In [3]:
obama_speeches=pd.read_csv('Obama Speeches')

In [4]:
len(obama_speeches)

519

In [10]:
nlp.max_length=100000000

In [5]:
obama_text = [text for text in obama_speeches.text[0:100]]
obama_text = ' '.join(obama_text).strip()

In [92]:
f = open('obama.csv','w')
f.write(obama_text) #Give your csv text here.
## Python will convert \n to os.linesep
f.close()

In [3]:
from textgenrnn import textgenrnn

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


In [4]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 4,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 40,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 15,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': True,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [88]:
len(obama_text)

1649213

In [7]:
file_name = 'obama.csv'
model_name = 'Obama'   # change to set file name of resulting trained models/texts

In [12]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 4-layer, 128-cell LSTMs
Training on 1,319,151 character sequences.
Epoch 1/15
1288/1288 [==============================] - 219s 170ms/step - loss: 3.0638 - val_loss: 3.0551
Epoch 2/15
1288/1288 [==============================] - 222s 172ms/step - loss: 2.4571 - val_loss: 1.5205
Epoch 3/15
1288/1288 [==============================] - 221s 171ms/step - loss: 1.4221 - val_loss: 1.2763
Epoch 4/15
1288/1288 [==============================] - 222s 172ms/step - loss: 1.2700 - val_loss: 1.1976
Epoch 5/15
1288/1288 [==============================] - 221s 172ms/step - loss: 1.2001 - val_loss: 1.1583
####################
Temperature: 0.2
####################
nd a standard and the policy of the problems and the strategy in the world. But what I have to make it is that the problem is that the programs are the country that we have to make sure that the problem is that the change of the world has been a both in the problem. It's time to turn the page on the

 do it to stand up t

he last new century affordable for all Americans of the war in Iraq and a state where people who can simply come together and stand up and the other day. We have to get the challenges we face are willing to take the benefits of the war in Iraq, but some of these kids are willing to see them from the

 failed to make their home to take the same choice in this country - or the same thing that we have to recognize that we have a chance to stay in the special interest in the streets. And I will make sure that the problem is the basic standard of the strategic policies of lobbyists that we can start 

e same of the most students who are the strength of this campaign – that says that we can still take a chance to start working at the country to give them the future of the reason to be a plan that will fight for the problem when we were written at the last seven years. The special interests of the 

####################
Temperature: 1.0
####################
ve always knew, I think it'g not be

In [3]:
file_name = 'obama.csv'
model_name = 'Obama1'   # change to set file name of resulting trained models/texts

In [4]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 20000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 150,   # set higher to train the model for longer
    'gen_epochs': 3,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 128-cell LSTMs
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training on 279,838 word sequences.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/150
273/273 [==============================] - 43s 156ms/step - loss: 6.3593
Epoch 2/150
273/273 [==============================] - 39s 143ms/step - loss: 6.0542
Epoch 3/150
273/273 [==============================] - 39s 142ms/step - loss: 4.8596
####################
Temperature: 0.2
####################
afford to be able to be able to take the same way that we ' re going to be able to be able to take a better tax cut that will be a family that ' s harder to be a sister who ' s been a sense of the united states . we have to make a difference in our country . we ' re going to play offense to play offense for the same kind of american people . and i ' ll also kno

of approach will be one school is tax black every college . and i ' ll keep my family to take the low - cradle to the sources of working americans who will be easy these differences spent . we have to accept that they new our world like so many system in our military 100 of troops - it ' s just hope in service before how many man suggested that i ' m phd , , you take them independents , who ' s not going to the crumbling practice , the extensively for brought the lobbying medical salary , the government must improve the failures of people are like but we have to take out of the situation we military on a sense of violence to the nominee . ( cheers , applause . ) is . and , applause . we have to have a way in this speech the kind of transition that christ clear on so many of black , all in a constitution legacy of the victims of america and families across long - neighborhoods . finally debating this party i ' t didn ' t ask for these tough hoped . well , in this debate , it ' s been a 

establish a universal health care plan that will cover every american a vehicle to the same kind of politics we need . and i won ' t just talk about the future of this generation . and when i am president , i will create a 21st century va to prevent the 21st century - american seek to meet the world . but there are no surprises . it is a privilege to be contained . senator mccain is not in the white house , we ' ve had to be focused on the long time . but we ' re going to talk about it . we ' ve got to talk about our values and our ideals . and we must also rebuild our standing on the streets of the americas . we ' ll have to keep pace the price of gas tax credits to the middle - class families . and i ' ll establish a new supportive services housing - - american community , and support , and the determination to be a part of american dream . we must have a party that runs prisons which lock an african - american community , and the non - of us to address the international community . 

273/273 [==============================] - 39s 143ms/step - loss: 3.3993
Epoch 8/150
273/273 [==============================] - 40s 145ms/step - loss: 3.1788
Epoch 9/150
273/273 [==============================] - 40s 148ms/step - loss: 2.9929
####################
Temperature: 0.2
####################
out . we ' re going to have to make sure that the american people are still the skills they need . and it ' s time to stop the fight for justice and bring the world ' s most dangerous weapons . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page on the arrogance and distractions and distraction , and agrees with the united states of america . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it ' s time to turn the page . it '

to find and common security throughout america . i don ' t want fema to be trade here now and to begin this back to this country . russert : right . . . . . obama : right , even . russert : when you may return " the first area in seeing medical students so students records work with service , commander - your support - - - if you replace you are willing to be getting sex essentially , but they ' re coming at the politics of the bargain . ( begin video clip ) unidentified male : i ' m sure , not only thought matthews : some change is a resolution to edwards at the same way that those consequences have skyrocketed you have interpreted to get new textbooks or the pension he made in persuade . when i came back to senator clinton , i found it because of some of those of these americans are being absolutely long as main street . so there are in the next election . we all have the technology we face on tv , as president of the united states senate will just be about where we might be less tha

...

the tools . it ' s time to honor tax incentives like the first part of our military is the time of us to write the oil and the diplomatic . my energy have grown by a bankruptcy bill that will be for them ' s time . my plan will invest in the housing and new orleans , and the training i ' ve met in this country know over two weeks . there was some of these ideas that we had in the past . russert : and what ' s on is the difference iraq is the very beginning . so we ' ve got to remember was the strength of a challenge that he says , he ' s honest about doing he ' s concerned . ( applause ) we ' re not going to sign on helping the countries that have some to do and have to say , " and so i think , a president the , will not be able to make a judgment . what i are looking for - that is true deal with how you ' re doing as you . you ' ll be the president and you ' re ready to raise back to them with make sure that the law - the ones we must stand ; before the one year you stand so the elect

universal health care plan that was well that the jobs can ' t afford . and it ' s an obligation to know that there ' s an american states for those who answer the most money we are as a result . the only thing i ' ll say i want is change . the way we have to make about progress is the end , but they are not just right to lead an end . the american people was the full time they already had a nation - support . " as i ' ve traveled to the congress still had to come so an black and not ask it what ' s possible for their country . you ' re a senior , who doesn ' t get our tax code wasted on the welfare agency . i don ' t want to see that . i don ' t want to see homeless veterans on the streets . i don ' t want to send another generation of american children to failing schools . i don ' t want that future , i ' ve been told you would say , but also only he ' s around this problem . he said , " it was my father that he was born . that ' s a " i have , that ' s not , going , here for the uni

and the mother can ' t get the chance to reach for their own , to ask for your standing , and for their rights and grandchildren . so we learned in the democratic country into a lot of time for american democrats and coming together forward , you ' re ready to turn the page on washington , and that ' s the america i ' m when we ' re working about to . we don ' t george washington is on us wants to go in up , who are young forgotten . they are working harder to be . today should be a hand for it . it is neighborhoods we ' ve lost - that when it ' s hard about the world when it was , the other best administration was that was young people to take on a path and their families , fighting to washington . it ' s with time . it ' s how we ' ve already in that moment . it ' s time for new leadership to make sure that our kids are safe - ever and opportunity ' s past , we put the jobs back the most of washington - then we ' ll make sure that all the american workers start being mine , what it d

is the way that we are still trying to heed the call of the united states . roberts : oh , you have really been pounded by the clinton trail . i know that bodes well for the november elections . stephanopoulos : so you believe the ideals will come behind , i said , " we all have an administration in america . it ' s about hard china that this generation has come up before our country want to walk away , ' to do their . . . ( laughter ) obama : . . . and that ' s the point that ' s within our grasp . so i ' m asking you to march with me , and work , and tell the country that stand before we before that you still have a politics that ' s been trying to drive with our troops , and we ' ve got to have some arguments . russert : but the time you face in washington , the and most game of them are being in the region . obama : well , i ' m not going to continue sort of the tit - off - tat . that is where i passed a first plan in the senate that will ' ve do , not the only - that act has been 

time , john mccain and hillary clinton also the people who do . and i believe that together is who that we don ' t have to wait in the world , that those ideals are challenging the chance to pass the power that will do not part , but there have never been in washington . russert : let me ask you , but let them just say that this is this job . i ' ve seen in the courts as a week , an living wage , how you will make that future . let ' s be the generation that that doesn ' t offer when you need to take look of this war in iraq as well . but i ' m the only candidate . i do not want the world - to make real its own political , and in twenty - years say , " or when the years of hope - that this country will be different . this is what we ' ve been in getting when i get making like the money of the first time since the most . time had a better life in their future . we can transform our cause and our cynicism and after the world ' s greatest threat . it ' s time for a foreign policy that say

In [3]:
file_name = 'democratic_nominees_text.csv'
model_name = 'democratic_nominee'   # change to set file name of resulting trained models/texts

In [8]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 5,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 20000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 150,   # set higher to train the model for longer
    'gen_epochs': 10,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [9]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 128-cell LSTMs
Training on 1,858,927 word sequences.
Epoch 1/150
1815/1815 [==============================] - 256s 141ms/step - loss: 4.4569
Epoch 2/150
1815/1815 [==============================] - 248s 136ms/step - loss: 3.7915
Epoch 3/150
1815/1815 [==============================] - 247s 136ms/step - loss: 3.6157
Epoch 4/150
1815/1815 [==============================] - 247s 136ms/step - loss: 3.5085
Epoch 5/150
1815/1815 [==============================] - 247s 136ms/step - loss: 3.4338
Epoch 6/150
1815/1815 [==============================] - 248s 137ms/step - loss: 3.3744
Epoch 7/150
1815/1815 [==============================] - 250s 138ms/step - loss: 3.3241
Epoch 8/150
1815/1815 [==============================] - 251s 138ms/step - loss: 3.2879
Epoch 9/150
1815/1815 [==============================] - 251s 138ms/step - loss: 3.2474
Epoch 10/150
1815/1815 [==============================] - 252s 139ms/step - loss: 3.2173
####################
Temperature: 0

and change is some different visions for that many . we talked about this , this is a means these principles i want to express it . i just believe we should send our faith together to every american , but also that everybody could pronounce my name . [ laughter ] " he was make these super pacs first time our records , and a moment in the way of developing the smart technology so that we don ' t have the luxury , progressives is also a advice that anger says it let ' s do that . i think the united states can offer as a consequence , but i ' m still out there looking for me . you have seen that the next one will change jobs with the most important anti - most , part of us . the future is more than i tired of an obama named ? john mccain is in washington that ' s costing me a pretty good understanding . you know , i was ought to be serious with all of you . audience members . four more years ! four more years ! the president . thank you so much , everybody . thank you . thank you ! thank 

i ' m tired of hearing about how to pay for college . and we ' re going to have to make a difference . and i ' m going to be able to do more to make sure that we ' re not going to let oil companies write this country ' s energy plan , and i ' ll never forget how she spent the final months of her life lying in a hospital bed , fighting with her insurance company because they claimed that her cancer was a pre - existing condition , and most importantly , i ' m not going to give up on the fight . i ' m not going to stand here and tell you that if we can ' t stop spending $ 10 billion a month in iraq , i ' m not sure that the democratic party has been talking about . it ' s not a sign - in - trade , and the world ' s best wealth generator - out - directly to the people who are struggling , and who ' s doing it , because i do think that the real mitt romney has embraced . i think it ' s important to remember that it was " tragic " to end " don ' t ask , don ' t tell " don ' t tell , " but n

and i quote , " i don ' t know what to do about ensures that we can ' t figure out how to fold them back . and i think that we should be able to find a job that pays a living wage , that ' s not change . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re fighting for . that ' s what we ' re going to have to work for it . we ' ve got a lot of work to do . and we ' re going to have to work for it . and we ' re going to have to make some tough choices . and i think that we ' re going to have to work for it . we ' ve got to make sure that we are 

, the good government can ' t engineer . that ' s what gathered in washington . it is why the decent country leads this nation together to compete with the skills that businesses are looking for right now . i believe that america is stronger when the differences and divided between the air and hope and cliff . this is the most important election of our lifetime . but we also believe we should fully recover no health care . and that ' s why i ' m certain we have to treat and walk into , nobody in india who has been telling us and it is still about exciting and , for your back once to move the country forward as americans , and an important record to but would get the economy growing up here in iowa / who wonderful faith and purpose are towards more competition . i believe in every american . and there ' s no doubt we can ' t figure out how popular they could be on this stage if you stand as at the unity of division . listen : " yes , i ' m laying out that we ' re going to try to divide 

...

la - based on the south side of chicago . and you know , i think that ' s a good thing . and if you look at the facts , it ' s no wonder that we ' re not even going to be deported from the only country she ' s ever called home . it ' s time to end the era of oil and natural gas , but also a lot of folks hurting out there . and we ' re going to be able to retire with dignity and respect . and the reason i ' m confident is because we ' ve got a different idea . you made that possible . you won ' t be able to make the mortgage payments . and we should strengthen medicare , and we can create millions of new jobs , if you want health care you can afford and education that helps your kids compete ; then i ask you to knock on some doors , make some calls , talk to your neighbors , talk to your coworkers , you can make it if you try . that ' s what we ' re fighting for . that ' s why i ' m running for a second term as president of the united states . audience members . four more years ! four m

el , and we ' re going to do . we ' ve got to make sure that we ' re not going to let it go bankrupt . " you ' re the reason why some outstanding soldier who started at the age of 55 , 000 in the last few months , we ' ve created a lot of people who are still struggling , but they ' re not going to be able to get the same kind of health insurance that members of congress get for themselves . we ' ll invest in preventative care and new technology to finally lower the cost of health care , not just to watch the tax credits that we ' ve been through together , we can ' t just cut our way in a balanced future . we ' re going to have to work harder . they ' re counting on now is that you ' re going to have to make some tough choices , but we ' re going to have to make some tough decisions about what we need to do to make sure that we ' re not going to be solved overnight . but we ' re going to do it in a balanced way . we ' re still going to have to make some tough choices , but we ' ve got

1815/1815 [==============================] - 257s 142ms/step - loss: 2.6625
Epoch 72/150
1815/1815 [==============================] - 260s 143ms/step - loss: 2.6585
Epoch 73/150
1815/1815 [==============================] - 258s 142ms/step - loss: 2.6518
Epoch 74/150
1815/1815 [==============================] - 260s 143ms/step - loss: 2.6490
Epoch 75/150
1815/1815 [==============================] - 259s 142ms/step - loss: 2.6440
Epoch 76/150
1815/1815 [==============================] - 258s 142ms/step - loss: 2.6397
Epoch 77/150
1815/1815 [==============================] - 258s 142ms/step - loss: 2.6374
Epoch 78/150
1815/1815 [==============================] - 275s 151ms/step - loss: 2.6315
Epoch 79/150
1815/1815 [==============================] - 259s 143ms/step - loss: 2.6269
Epoch 80/150
1815/1815 [==============================] - 259s 143ms/step - loss: 2.6229
####################
Temperature: 0.2
####################
el , the world has constantly been more dangerous than the last 

s a u . s . citizen make a living child in poverty that pay their bills ; that ' s the last time in richmond , i ' m going to build a strong initiative in global regions and home values and the hopes of all americans . we ' re going to do better and that the american people don ' t have to tell them about my words . college ' s the problem last $ 250 , 000 tax cut for those at the top in the very during that crisis . we don ' t have a system . you ' re saying we ' re sorry , to get the support she needs , kids in nursing homes for life if they medicare and respect for all – defends the hard people in friends of the needs and our energy dependence on foreign oil and invest in renewable sources of energy . wind and solar , biofuels and wind turbines and the auto industry , if we provide tax breaks to companies that are investing here in ohio , right now - - give out to a nation of laws they need to just go backwards to the theory that we ' ve done , we ' ll win iowa again . we will win t

KeyboardInterrupt: 

In [10]:
file_name = 'dem_text.csv'
model_name = 'dem_nominee'   # change to set file name of resulting trained models/texts

In [11]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 20000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 100,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=2048,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 128-cell LSTMs
Training on 789,656 word sequences.
Epoch 1/100
385/385 [==============================] - 112s 292ms/step - loss: 5.7809
Epoch 2/100
385/385 [==============================] - 110s 285ms/step - loss: 4.5407
Epoch 3/100
385/385 [==============================] - 111s 288ms/step - loss: 4.2040
Epoch 4/100
385/385 [==============================] - 112s 291ms/step - loss: 3.9851
Epoch 5/100
385/385 [==============================] - 111s 289ms/step - loss: 3.8199
####################
Temperature: 0.2
####################
you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you so much . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank y

found no age . that ' s his late days for mine . it didn ' t he is enough . but there is a whole host of jarring - national powers - and the people - old at the spirit that the federal level , the disparity who has brought the help for a middle - class education that we did for the most women who serve that we are a president who has imagined a different . one of the things i found in tonight or america around the country , and gathered through our doorstep . recently , the first day close al qaeda has lost nearly 100 million new jobs in 5 new people . and so , . . 38 million people , he gave more men the chance . parents marched on their feet . when people stood somewhere else , you ' ve thought it in some different ways in life in the last ten years without davenport . did you have with the tools . the only poet regional support has brought full of the new threats . we need to take our bullets out . i ask you to favor what you should give past stand ? you want to tell you what i woul

to do . i ' m not going to be a president , i ' m not going to be able to get there . but the truth is , i ' m asking you to be your partner with the nra of gun violence . and i will work hard to all these people who are working with untreated jobs and benefits . i also believe that the american people are owed to the choices we face . i think it ' s a fault that denies my grandfather to shut down the corporate tax cuts that actually came into the pockets of working americans . we ' ll create a tax credit that makes a difference in the american economy . and i think there ' s a lot of different stories about what ' s happening in this election . so i ' m going to stand up to the republicans , and i think that ' s a good thing . i ' m asking you to wake up . i ' m asking you to wake up on the ballot . i ' m running for president because i want you to know that i am excited about . and i think we have to do this . i have a very clear experience of my campaign . i think , i think it ' s i

385/385 [==============================] - 109s 282ms/step - loss: 3.1899
Epoch 12/100
385/385 [==============================] - 110s 285ms/step - loss: 3.1173
Epoch 13/100
385/385 [==============================] - 111s 288ms/step - loss: 3.0497
Epoch 14/100
385/385 [==============================] - 110s 287ms/step - loss: 2.9875
Epoch 15/100
385/385 [==============================] - 110s 286ms/step - loss: 2.9328
####################
Temperature: 0.2
####################
want to extend the honor of the teaching , and the opportunity to reach for those who are rich and poor , black and white , latino , asian , and native american , wherever they ' re doing so that we can ' t be bogged down . we ' re going to invest in the middle class , working families , and homeowners who are struggling to join up to their god - given potential . and we ' ll have to remind ourselves that we rise and fall together . and we ' re going to make sure that every single child has a chance to work with a

will have no idea that his record is clear , it ' s not yet wrong . although it ' s obviously very unfortunate network to do nothing to sort out a issue to a time we did to keep sending them the " president of central american military is to cause again . our congress also fighting out every fourth world continues to build an america , we should create a new direction , we have to come up to one every step to get it done . dickerson : i can start even with what i expect to do , but let ' s be new short - term , cutting a part of how we — as a counter and terrorism , we should be made a stable enemy in the face of that weapons and new ties with other areas in the region that put against isis . as president , i will respond that our forces will persevering and go after the war must come together in a time , only how when it comes to housing will not clean the air environment for the future . if you can believe that every american child shared the vote will lead . and i want to listen wit

...

that ' s not true . the economy that faces a secret and for the companies that direct the most of health insurance for americans , and energy company in the long term hours . he knows he can ' t get done because it can ' t control without spending money . because we need to raise teacher salaries , we will protect every child ' s plan to compete in the global marketplace . they need to create a mortgage of reform ; they will encourage any government that needs to be made a political strategy on judgment . russert : the u . s . presence in iraq , and an rising tide of special destruction . that ' s why we need to make this phone calls again .   i really think i ' m against too many folks in the next nine talk . about fighting the issue of our nation . it ' s the legacy of the lives of my being . " now , let ' s give it to our future . doing so back the debate , i hear you , the people i put a little more difficult . it doesn ' t get change by special working - on work , so that america 

honored to serve as president . so we are going to fool those who voted for what i believe and to do something that we can try to make sure that we don ' t . we ' ve learned that politics . in the middle east , we learned that when the folks were in ; you know , the oil companies , which ' s the problem we ' ve got to teach young people and families to have trouble in trouble . i knew that i was getting the eyes of a white man and i tried to go to work to make sure that i can work with you and your family . i ' m not going to let anyone decide that i thought we ' re going to do something that we can ' t stand up for wages , that we ' re not leaving to . we ' re going to raise taxes on the wealthy people . i ' m not running to challenge the way or loyalty we need . so i want to thank all of you for coming " to all those who have the best hopes of america . some of the fundamental differences that we did in illinois just as a nation , it ' s a sign of leadership . and i am running for pr

tell you that , i will never sit up until i was a young lawyer . so i was really pleased that i could have a clean line in the house of iowa . and i ' m very grateful for that . i am so flattered and obviously , you know , i ' m not in this campaign . stewart : ok . clinton : i haven ' t changed the fight for donald trump ' s third advantage of the union , he said , ' well , i guess , i think that ' s a lot of talk about what is on the minds of our friends , and we ' re going to defend it . we ' re ready to play offense for organized labor . it ' s time to honor the service of my generation . i have seen our nation ' s word , and i ' m proud that we ' ve always been talking about here in the country . i want you to know that i ' ve been speaking for the creation of the national baby . i live in the promise of a great teacher coming out there . that ' s why i ' m in this race . i am running for president because i am sick and tired of democrats thinking that we ' re having a better and 

taxes for the wrong choices and the wrong choices . on tuesday , i believe that every family head in every ' ' s bankruptcy bill . it may mean most of $ 14 , 000 a year to enter the oil in poverty . now it would even tell the state that would promote the federal government to stop , stop agreed to the iraq . and that puts some 55 , 000 of course we believe that what led to a crisis of troops as a head to a good good or work . and our goal too is making . our energy is too great when our and that it comes to power and playing it . and that ' s why i believe we need to end a war in iraq a new - - community occupation , and try to turn this into an international effort to come up to a time of removing in you . you don ' t know how you ' re going to get sick or not only as high as she can do to succeed in this campaign and try to defend our country . but if you ' re ready for change , then i ask you to have another challenge - - on what i think is the opportunities for me , and i am thrill

been asking for your help . and if you get ahead and we know what you ' re doing . thank you . [ applause ] and i want to be a good job . but i also want to thank my wonderful friend mayor mayor arms . today , i saw a man here in indiana , when i saw the secretary of 2012 , senator obama , senator mccain has been consistent with the fact that we ' re still going to solve common sense of the real issues that climate change hold us back . we need to build a strong military military , and a world must be a beacon of peace . and we must never forget once again for the sake of our children . i am pleased to joe who asked me to have to raise the money and paying the cost of health care by the congress , for example , over $ 1 , 000 a year . it is also going to give people a chance to make a college living up . we have to get serious about investing in childcare , alternative fuels and transition that business and investments in renewable sources of energy and advanced manufacturing . we ' ll

In [2]:
file_name = 'dem_text.csv'
model_name = 'dem_nominee1'   # change to set file name of resulting trained models/texts

In [3]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 5,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 150,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [4]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=2048,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 5-layer, 128-cell LSTMs
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training on 789,822 word sequences.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/150
385/385 [==============================] - 112s 290ms/step - loss: 5.9660
Epoch 2/150
385/385 [==============================] - 107s 279ms/step - loss: 4.5366
Epoch 3/150
385/385 [==============================] - 108s 280ms/step - loss: 4.1471
Epoch 4/150
385/385 [==============================] - 108s 280ms/step - loss: 3.9138
Epoch 5/150
385/385 [==============================] - 107s 279ms/step - loss: 3.7441
####################
Temperature: 0.2
####################
to have a good job . i ' m going to have a president who ' s going to look at the failed policies that we can do . and i ' ve seen that in the face of the world . the fact that we ' re not

that bridge about the cause was whistle . they really say , " the reason she had to come as a housemaid . she is the kind of president , who brought us a generation safely , and for coders , not just those meaningful priorities , but not those social gospel . i want to speak directly about afghanistan from iran emerging embassies and plutonium . when our commitment is that this is a job business . and it ' s a plan to make big prosperity . it is a power who will cut waste . i will offer universal health care to taxes and make it independent . my president has to say if nothing concerns about taxes , they didn ' t go some insurance because of the companies . so we ' ll funding our teachers and we want health care that keep tax incentives for less jobs while bad businesses were falling behind , and directly to teach them and build an international movement , then will slow review ' s nuclear facilities . make sure in strategy , are joined the winning . clinton : you know , that is the wa

he ' s at the same time . he ' s a steelworker , and he ' s a steelworker . he ' s a steelworker . he ' s seen the biggest tax cuts for individuals and the pockets of working americans . and i believe that we need to keep america safe . i want to keep the pressure of the international support of our democracy . we must oppose the goal of afghanistan and innovation and alliances – and restore our alliances , and make our schools competitive , and our economy also , and our economy , and our workers , our workplaces , our society , our workplaces , and the character of our nation . and we will invest in our infrastructure and our schools , our roads , our bridges , our tunnels , our ports , our airports , our ports , our airports , our water systems , and our society , our selfish , our nation , our nation , our nation , and our nation ' s sake , and our greatest monument in the world . we ' re going to help small businesses determine who ' s just getting their workers . and i ' ll also 

385/385 [==============================] - 107s 277ms/step - loss: 3.1254
Epoch 12/150
385/385 [==============================] - 107s 277ms/step - loss: 3.0546
Epoch 13/150
385/385 [==============================] - 107s 278ms/step - loss: 2.9904
Epoch 14/150
385/385 [==============================] - 108s 279ms/step - loss: 2.9320
Epoch 15/150
385/385 [==============================] - 108s 281ms/step - loss: 2.8820
####################
Temperature: 0.2
####################
el . ( applause ) ( cheering and applause ) unidentified female : hello , thank you . clinton : thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . thank you . tha

john mccain . ; if you look at a politics - - are a woman ' s name , about everyone everything she ' s proposing a lot you can support in alzheimer ' s premiums . and to work with $ 1 billion in the state department in this race . too here ' s because i believed in this race . i feel sure the only we need to ask them is not not the words , but its future we are against a discovery where everyone can grow the world - on building or unwilling special - military label . the major government should also allow resources to develop our own future . instead of letting citizens know what ' s hard for america ' s work . it ' s about everyone that people have become tested in of so many who may remain heard ; leaving people to be heard , please to see iranian obligations victory . half - thirds of staying so much water at record they live to treat . and when he worked for him , then i would get done everything they had to . and who wouldn ' t raise the minimum wage and the right kind of top - ri

...

, from the way to make something happen . so let ' s make clear ourselves . we all know that this is a challenge of our economy – it ' s what our country has built . and that man has made money so much of his hands , and we also have done what ' s right for us , we are going to enforce the rights of law and respect to make sure that the family of the american people are living up in the process , and they are making the same case in our country . and we need to bring our country together . we have to heal these divides in the rest of the american people . and we ' re going to take a hard look at what we have to do to make sure that we are working together . and that is why i ' m proposing a message to re paying our economy into education . that is the kind of change we need to take . we passed a war in iraq that would have no ongoing , in the bad campaign . it isn ' t just a problem , it will not be easy . [ applause ] — and we ' ve got to try to make sure that we are working for doing

, to organize the men and women of women who fight for freedom and organize for freedom around our nation . and we ' re going to be stronger together . we reject the threats of the nation in the nation : the challenge of our economy , the price of the future – our future should be strong when we built together . we can ' t be the country where the united states of america ' s most six six years . we ' ll invest in health care , and we ' ll use the political process of giving tax breaks to companies that ship jobs overseas , invest in the middle class , and the wrong direction , and we ' re ready to change it . we ' re going to bring back our economy . we ' re going to cut taxes for companies making the career of ordinary americans . finally , we ' re going to put back our place – and we could do it with fear that only put together in an america that gives the world behind its next dream . yes , let ' s be clear . i want to thank my friend , for giving me the honor to join me in a lot o

but we are far all the , we are our kids . and we ' re not going anyone to do thinking about issues that are leaving our policy and doing what i can do . and i think we need a lot of smart challenges that are going to be done in an election - in the way i think that the issue is . are that the sound way in this election is not between regions or religions or genders . it ' s not about rich versus poor ; young versus old ; and the american people . he ' s part of a broader view of george bush has failed that no one restrictions on ever the next prime minister that he wanted us to call the press , you don ' t do it if you ' ll be it today that is my debate about the american people . today , the oil companies got help that keep our growth overseas . for all small breaks for our families , and our workers become going never again . too many without any programs we haven ' t taken . and this has caused it to turn people on top of our voters . and certainly , if the united states is put thr

of course , it ' s worth really too , think what if it was coming to catholic church and which begin to remember the democrats on behalf of us . look , just and that couldn ' t work without me . if i stand with the big jobs we ' ve been and to expand , and the ones that she has , about the [ inaubible ] , he was always making it clear that if you ' ve got a life , there are no republicans who ' d say about more americans or who will stay in that white . we should fight for prosperity . i believe a promise for our children is in work . the middle east is already moving into wisdom . the war in iraq is just whether we standards . we can do that . we also know the truth . don ' t mean that our fiscal african - american african - american americans can be put on poverty - and too much . and the american people got a hand west - so young people could be in poverty . getting by time to get our leadership home . we need a president who sees them from the same kind of life . i understand what 

que . if you listen to value affordable and tuition - - for the first time in history , this is the kind of leader i intend to be . and finally as i ' ve said before , i brought them back to vote . you know , you ' re pretty hard , you ' re tired of being , you know , activities support young people , whatever is people going to be able to get there ? and sometimes , you know , they brought a government to make america ' s own power . it ' s time to heed the politics of a politics that is nothing right . and i think it ' s time to restore the ban of oil failed and ensure that we send our workers special interests into a company that won ' t afford another family ' s health care program . it ' s time to bring the drug drug playing the middle class , and we are going to move it forward . we ' re going to begin – because we have to do more , then we have to get people together to get to work , and to clean energy , and the choices to make it more secure and secure . that ' s the kind of p

In [5]:
file_name = 'dem_text.csv'
model_name = 'dem_nominee2'   # change to set file name of resulting trained models/texts

In [6]:
model_cfg = {
    'word_level': True,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 6,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 10,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 20000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 200,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': False,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [7]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 6-layer, 128-cell LSTMs
Training on 790,026 word sequences.
Epoch 1/200
771/771 [==============================] - 124s 161ms/step - loss: 5.8311
Epoch 2/200
771/771 [==============================] - 122s 158ms/step - loss: 4.4415
Epoch 3/200
771/771 [==============================] - 122s 158ms/step - loss: 4.0855
Epoch 4/200
771/771 [==============================] - 122s 158ms/step - loss: 3.8601
Epoch 5/200
771/771 [==============================] - 122s 158ms/step - loss: 3.6978
####################
Temperature: 0.2
####################
. . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

thank you . thank you , god bless . thank you . ( applause ) over so many of you from massachusetts . my wife continued post patton cross at the army air filtration plant . he was — you couldn ' t assess it . i run the privilege for some folks that have benefited out of service . i ' ve really met with political alabama decisions , a lot of meeting with race . there are reports from miami echoed libraries . faith , part of all you ' re here today . and the time they come to the other side , but if it was met them , we could be surprised to go to forth . but iraq becomes tried to provide a war to - iran . be enough to help support say that government continues to do more about this future . extend our economy , in the future for new vote , congress won ' t appreciate the sons and daughters of students whose faith . and also for those whose saying , some of you see losing it up . i represented our office tonight tonight . we will bump write , and remain changes to make sure they have no 

has served our country and not only for the american dream , but the potential of the american people . and we know that . and we ' re going to be doing something about what we ' re going to do to bring about our troops and the world . and we must tap the world that are set up to all of us . we need to lead the next generation of biofuels . there are no more important incentive to be in the senate . and what you ' ve said is an attack in the book of the united states . this is your moment . it ' s been a battle for the men and women who marched and struggled and cleared our country . and in the end of my first term , i will not invest in fear . i will not privatize social security . i will not cut benefits for the middle class . i believe the american people deserve better than you have to make in this election . and if you ' re working with this , and you can do what they want to do . we ' re going to close the loopholes that pay for everyone who make more than $ 200 , 000 a year . an

proud to be joined here today . i want to thank your friend , mayor mike , who spoke about the issues of the people , and i ' m in this race , and i ' m running for president because i believe that we can make progress in the way of the american people . and i ' m glad that we can ' t afford to settle for a massive recession . and we should be leaving the kind of future we want . we ' re going to have a president who ' s taken a gun - mccain ' s approach to address iranian influence . we ' re ready to play offense for universal health care . it ' s the wrong choice for the future of america . and we ' re ready to play offense for working families . i ' m ready to play the hard work for working families . we ' re a country of optimists . we ' re the can - do people . and we ' re usually going to have a president who ' s taken a gun - mccain ' s approach to address iranian influence . we should maintain the terrorists and deployment of iraq . i will maintain the promise of the past , and

of calm family inequality – the same electoral bill that ' s worked on because i ' m standing up against the lgbt americans who make that choice ' s right on what happened in a way should be released . we had taken the toughest vote of the hour after what we ' re going to do : it ' s really what they are – and , and brought them down when he ' s leading . i think you have a track to get there . all these young people have spoken to the affordable and benefits of living . and then when he ' s said , things , it ' s been one of the tasks coming of the energy crisis . against climate change , we will invest $ 500 60 billion over the next ten years – congress really have less right now . and i ' m confident that no child , you have every one where i heard you in this september debate . clinton : right . yes . thank you for being here in iowa . clinton : you know , domestic , your mom has always been in the front - year old life this health is not to see both money , but once again , they d

...

there ' s a new epidemic of struggle and effective and to should ensure that america is beyond the right side in history . we can work to core – need to protect and protect the muslim world ' s fighting for human rights progress , none of us know that about something . you can go to what you ' re going to do and will get the people . and i especially got to tell them , are a gun , tough hearing and in the second place have ever made a mistake . russert : one of the differences you had some , you know , one of these i did when it comes to my own tough law with kids to kids in a school . and i want to — very well clearly i can ' t do this without your help than it ' s for . we need to restore the end of our future , created deep - based , and in return - the cost of everything we ' re doing . we ' ll be the right against every single presidency – it is the most political election our country will look in 2000 . i ' m proposing that you have the opportunity to participate in the years bef

, i ' ve been traveling with my dad . i ' m now for our best education . we ' ve got to get a education to get to know what - many fathers can be really living by these parents . the government will be forgotten by retirement . but we have to be looking back and there are some real bases there . well , here is not another issue . this election is our policy ; the call for blame , the ideals that i was not with a tragic war . it was a lot building a company that has been run on their commitment . it ' s face that we make a quality education . and then — let ' s say yes to good education . there are two kids , introduced and had not had that . and that ' s why , there ' s something we can ' t achieve . . . the government really has cost us less than just the tax cut in the year . then , for the company , it ' s going to be much easier to make it a little more than anything . so we can build an economy that ' s stronger at home and respected in the world . i ' m a country that hasn ' t gi

a bridge looking through the tough choices . you know , we ' re the technology of students ' s first bad - term interest . i just ' ll oppose the tax code that was so different about bill at first time : it is about their lives . clinton : it ' s great . are you making a hard honor job and then you want to recognize what it ' s . all you haven ' t , we ' ve been doing calls the same kinds of arguments that we ' ve had in many years , over the — as one of these biggest changes , two weeks ago , i left my career to the doctor because he will keep donald ' s third fighting the only economy for seniors for families who need good to industry . and we ' ll ask companies to join – use cuts and pay them to pay them . it ' s washington ' s total it ' s , we ' re not spending $ 150 billion at the top overseas they did have gone off with her . yes , what i ' m hoping to do is you , to get to talk to people who are living with their child . working people are being left out in november . so when i

still be kind of . . . it was in iraq . . . . four months , this administration change fresh - - and that brings democrats in the region . and this administration ' s time of american workers have already worked , and not being those organized as they were asked , and that was so different . they don ' t feel the cause of america . we are making this pledge . that ' s why we ' ll create a clean market that can cut every dollar and provide a new price for a tax cut back child from the most reason . my plan will cut costs of those working families . i would pay much of them and money from them to pay their fair share . and he has no idea in the campaign . that ' s why is what this can try to happen ? you will be able to say a big view — if it were hard . not after the end of the long term we could keep middle - class families a right . we ' ll give you the insurance to those new costs , start to college , and the congress says , ' ' re wrong ? ' ' healthy forests ? ability to work , you 

no more , then i believe we are going to be reaching out , not having under the ballot . ' we are all of the kind of skills . i ' ve never gone into strong at the first level that our daughters and grandparents do . there ' s something wrong for when one election of love you can see about yes , that you should be able to say you have a part , and making america a good partner , and this country will create an economy where we keep our national debt by supporting teachers and tax relief that would help create more fuel efficient cars and we get the deficit in health care — working families by their neighbors to buy a house - class education . the first jobs this crisis is made $ simple billion to the wealthy that ' s falling away from corporations to put jobs overseas back - and - dollar interest on the tax code . and , they won ' t make social security to fill , they can ' t extend a larger price of their nuclear approach to get out . there will always quickly be given me with another 

In [1]:
file_name = 'dem_text.csv'
model_name = 'dem_nominee3'   # change to set file name of resulting trained models/texts

In [2]:
model_cfg = {
    'word_level': False,   # set to True if want to train a word-level model (requires more data and smaller max_length)
    'rnn_size': 128,   # number of LSTM cells of each layer (128/256 recommended)
    'rnn_layers': 4,   # number of LSTM layers (>=2 recommended)
    'rnn_bidirectional': False,   # consider text both forwards and backward, can give a training boost
    'max_length': 40,   # number of tokens to consider before predicting the next (20-40 for characters, 5-10 for words recommended)
    'max_words': 10000,   # maximum number of words to model; the rest will be ignored (word-level model only)
}

train_cfg = {
    'line_delimited': False,   # set to True if each text has its own line in the source file
    'num_epochs': 100,   # set higher to train the model for longer
    'gen_epochs': 5,   # generates sample text from model after given number of epochs
    'train_size': 0.8,   # proportion of input data to train on: setting < 1.0 limits model from learning perfectly
    'dropout': 0.5,   # ignore a random proportion of source tokens each epoch, allowing model to generalize better
    'validation': True,   # If train__size < 1.0, test on holdout dataset; will make overall training slower
    'is_csv': True   # set to True if file is a CSV exported from Excel/BigQuery/pandas
}

In [ ]:
textgen = textgenrnn(name=model_name)

train_function = textgen.train_from_file if train_cfg['line_delimited'] else textgen.train_from_largetext_file

train_function(
    file_path=file_name,
    new_model=True,
    num_epochs=train_cfg['num_epochs'],
    gen_epochs=train_cfg['gen_epochs'],
    batch_size=1024,
    train_size=train_cfg['train_size'],
    dropout=train_cfg['dropout'],
    validation=train_cfg['validation'],
    is_csv=train_cfg['is_csv'],
    rnn_layers=model_cfg['rnn_layers'],
    rnn_size=model_cfg['rnn_size'],
    rnn_bidirectional=model_cfg['rnn_bidirectional'],
    max_length=model_cfg['max_length'],
    dim_embeddings=100,
    word_level=model_cfg['word_level'])

Training new model w/ 4-layer, 128-cell LSTMs
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Training on 3,688,090 character sequences.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/100
3601/3601 [==============================] - 654s 182ms/step - loss: 2.4102 - val_loss: 1.3081
Epoch 2/100
3601/3601 [==============================] - 648s 180ms/step - loss: 1.2542 - val_loss: 1.1574
Epoch 3/100
2155/3601 [================>.............] - ETA: 3:34 - loss: 1.1713